In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

In [ ]:
X_train = pd.read_csv('../input/to-the-top-v2/X_train.csv',index_col=[0]).reset_index(drop=True)
X_test = pd.read_csv('../input/to-the-top-v2/X_test.csv',index_col=[0]).reset_index(drop=True)
y_train = pd.read_csv('../input/to-the-top-v2/y_train.csv',index_col=[0])

In [ ]:
xgb_params = {
    'eta': 0.0202048,
    'booster' : "gbtree",
    'max_depth': 5,
    'subsample': 0.6815,
    'colsample_bytree': 0.701,
    'objective': "binary:logistic",
    'eval_metric': 'auc',
    'verbosity': 1
}

In [ ]:
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc = np.array(auc, dtype='f')
    auc /= (nfalse * (n - nfalse))
    return auc

In [ ]:
from sklearn.model_selection import KFold

fold_score = []
kf = KFold(n_splits=10)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_train)):
    dtrain = xgb.DMatrix(X_train.loc[train_idx], y_train.iloc[train_idx])
    dtest = xgb.DMatrix(X_train.loc[test_idx], y_train.iloc[test_idx])

    num_boost_rounds = 560
    # train model
    model = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds)
    fold_score.append(fast_auc(y_train.iloc[test_idx],model.predict(dtest)))
    model.save_model('model_fold_' +str(fold)+ '.json')

In [ ]:
print('Cross Validation score = %1.6f' % np.mean(fold_score))

In [ ]:
result = []

for fold in range(10):
    model = xgb.Booster()
    model.load_model('model_fold_' +str(fold)+ '.json')
    result.append(model.predict(xgb.DMatrix(X_test)))

In [ ]:
submission_df = pd.read_csv('../input/santander-customer-satisfaction/sample_submission.csv')

In [ ]:
submission_df['TARGET'] = np.mean(result,axis=0)

In [ ]:
submission_df.to_csv('submission.csv', index=False)